# Постановка задачи

Тему с RNN не пустил в ход, т.к. надо переделывать sample и batch конвертеры на RNN - добавлять еще  один дименшн.
По хорошему надорефаакторить в отдельный компонент и импортировать его в плеер и в агента.

# Импорты

In [1]:
# Системные импорты и настройки
import os
import sys
import yaml
import random
import warnings
import ipynbname
import logging.config

warnings.filterwarnings('ignore')

# for local development
RT_LIBS_PATH = "/Users/alex/Dev_projects/MyOwnRepo/rt_libs/src"
BA_LIBS_PATH = "/Users/alex/Dev_projects/MyOwnRepo/basic_application/src"
sys.path.append(RT_LIBS_PATH)
sys.path.append(BA_LIBS_PATH)

# read config
with open('config.yaml', "r") as stream:
    config = yaml.safe_load(stream)
    
# set logging config
log_config = config.get("log", None)
logging.config.dictConfig(log_config)

# set notebook alias
ALIAS = ipynbname.name()
print(ALIAS)

gen12.1-rnn-Abstract-02-CloseSignal


In [2]:
# DS frameworks
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns

%matplotlib notebook

In [3]:
# NN Frameworks
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, LSTM, Dropout, Concatenate, BatchNormalization
from tensorflow.keras.layers import Conv1D, MaxPool1D, AveragePooling1D, Flatten
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.python.keras.models import load_model, clone_model

devices = tf.config.list_physical_devices()
print(devices)

2023-06-20 23:37:54.440486: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]


In [4]:
# RT packages
from rl import DQNAgent
from env import TradeEnv
from core import ConstructorGen2

from data_point import DataPointFactory

from train_tools import Player, plot_and_go

from train_tools import Player, plot_and_go
from train_tools.train_plot import TrainPlot4
from train_tools.train_manager import TrainManager

In [5]:
seed_value= 0
#os.environ['PYTHONHASHSEED']=str(seed_value)
random.seed(seed_value)
#np.random.seed(seed_value)
#tf.random.set_seed(seed_value)

# Конфиг

In [6]:
observation_len = 1

# Параметры точки наблюдения
observation_config = {
    "observation_len": observation_len,             # Количество точек наблюдения в сэмпле
    "offset": observation_len,                      # Количество точек наблюдения в сэмпле
    "future_points": 0,                             # Количество будущех точек для предсказания тренда (временное решение)
    "step_size": 1,                                 # Шаг по датасету
 }

# Датасет

In [7]:
np.random.seed(2)

n_steps = 100
sample_num = 10
sample_size = n_steps//sample_num
safe_interval_size = 3

dataset = np.empty(0)

sample_size_ = sample_size - safe_interval_size
for i in range(sample_num):
    safe_interval = np.zeros(safe_interval_size)
    total = 0
    while not total==1:
        sample = np.random.uniform(size=sample_size_) > 0.9
        total = sum(sample)

    dataset = np.concatenate([dataset, sample, safe_interval])
    
data_train = pd.DataFrame(dataset, columns=["close_signal"], dtype=np.int8)

#data_train['close_signal'] = data_train['close_signal'].replace(0, -1)
#data_train['close_signal'] = data_train['close_signal'].replace(1, 0.5)
#data_train['close_signal']  = data_train['close_signal']  - 0.5

print(data_train["close_signal"].sum())
print(data_train["close_signal"].count())

10
100


# Инициализация компонентов

## Datapoint factory

In [8]:
dpf_train = DataPointFactory(dataset=data_train, **observation_config)
dpf_test = DataPointFactory(dataset=data_train, **observation_config)

## Env

In [9]:
core_config = {
    "context":{"class": "Context", "params":{}},
    "action_controller":{
        "class": "AbstractCloseSignal",
        "params":{ 
            "penalty": -2, 
            "reward": 0,
        },
    },
    "observation_builder":{
        "class": "ObservationBuilderInput",
        "inputs": [
            {"class": "Input1D", "features": [
                {"class": "TradeStateFeature", "params": {}},
                {"class": "RawValueFeature1D", "params": {"name": "close_signal"}}
            ]}
        ]
    }
}
# = = = = = = = = = = = = = = = = = = = = = = = = = = = = =
core_constructor = ConstructorGen2()
env_core = core_constructor.get_core(core_config)

# train environment
env = TradeEnv(env_core, dpf_train, alias=ALIAS, log=False, log_obs=False)

# Нейронная сеть

In [10]:
ACTIVATION = 'relu'
def create_q_model(env):
    num_actions = env.action_space
    #----------------------------------------------
    
    inp_static = Input(shape=(1, env.observation_space[0]))
    classif = LSTM(8, activation=ACTIVATION)(inp_static)

    output = Dense(num_actions, activation='softmax')(classif)

    model = Model(inputs=inp_static, outputs=output)
    return model

model = create_q_model(env)
model_target = create_q_model(env)

print(model.summary())

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1, 2)]            0         
                                                                 
 lstm (LSTM)                 (None, 8)                 352       
                                                                 
 dense (Dense)               (None, 4)                 36        
                                                                 
Total params: 388
Trainable params: 388
Non-trainable params: 0
_________________________________________________________________
None


2023-06-20 23:37:59.570122: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Обучение

In [11]:
next_state, reward, done, _ = agent.env.step(0)

NameError: name 'agent' is not defined

In [ ]:
state_tensor = agent._sample_transformer(next_state)
state_tensor = np.expand_dims(next_state, (0,1))
state_tensor.shape

In [ ]:
state_tensor

In [12]:
"""
Реализация DQN-агента.
В этой верси все обновления (тренировка сети, параметры алгоритма) привязаны к фреймам.

"""
import time
import random
import logging
import numpy as np
import tensorflow as tf
from datetime import datetime
from collections import deque

from rl.components.replay_buffer import ReplayBuffer

logger = logging.getLogger(__name__)


class DQNAgent:
    EXPORT_FORBIDDEN_ATTRIBUTES = ("model", "model_target", "optimizer")

    def __init__(self, env, model, model_target):
        super().__init__()
        self.env = env
        self.model = model
        self.model_target = model_target

        self.state_size = env.observation_space
        self.action_size = env.action_space

        # decay or discount rate: enables agent to take into account future actions in addition to the immediate ones, but discounted at this rate
        self.gamma = 0.99

        # Number of frames for exploration
        self.epsilon_random_frames = 5000
        self.__epsilon_greedy_frames = 100000

        # Epsilon greedy parameter
        self.epsilon = 1
        self.__epsilon_min = 0.01  # Minimum epsilon greedy parameter
        self.epsilon_decay = self.__get_decay()

        # # Experience replay params and buffers
        self.__max_memory_length = 100000
        self.replay_buffer = ReplayBuffer(self.__max_memory_length)

        self.batch_size = 32
        self.update_after_actions = 4

        # update every N frames
        self.update_target_network = 1000

        self.max_steps_per_episode = 10000

        # episode params
        self.state = None
        self.episode_start = 0
        self.episode_reward = 0
        self.episode_loss = []

        # train stat params
        self.frame_count = 0
        self.episode_count = 0
        self.running_reward = 0
        self.running_loss = 0

        self.max_reward_length = 30
        self.episode_reward_history = deque(maxlen=self.max_reward_length)
        self.episode_loss_history = deque(maxlen=self.max_reward_length)

        # self.learning_rate = 0.00012  # rate at which NN adjusts models parameters via SGD to reduce cost
        self.loss_function = None
        self.optimizer = None

        self.new_episode = True

        np.random.seed(0)

    def reset(self):
        pass

    @property
    def epsilon_greedy_frames(self):
        return self.__epsilon_greedy_frames

    @epsilon_greedy_frames.setter
    def epsilon_greedy_frames(self, value):
        self.__epsilon_greedy_frames = value
        self.epsilon_decay = self.__get_decay()

    @property
    def epsilon_min(self):
        return self.__epsilon_min

    @epsilon_min.setter
    def epsilon_min(self, value):
        self.__epsilon_min = value
        self.epsilon_decay = self.__get_decay()

    def __get_decay(self):
        return (self.epsilon - self.epsilon_min) / self.epsilon_greedy_frames

    @property
    def max_memory_length(self):
        return self.__max_memory_length

    @max_memory_length.setter
    def max_memory_length(self, value):
        self.__max_memory_length = value
        self.replay_buffer = ReplayBuffer(self.__max_memory_length)

    def _sample_transformer(self, state):
        if isinstance(self.env.observation_space, list):
            # 2D samples
            return list(map(lambda p: np.expand_dims(p, (0,1)), state))
        else:
            return np.expand_dims(state, (0,1))
        

    def _batch_transformer(self, batch):
        """работает для 1D и 2D батчей. После zip(*batch) получаем списки из фичей по размерностям"""
        if isinstance(self.env.observation_space, list):
            # multiple input
            return list(map(np.array, zip(*batch)))
        else:
            # single input
            shape = np.array(batch)[0].shape
            output = np.array(batch).reshape(-1, 1, *shape)
        return output

    def act(self, state):
        # Use epsilon-greedy for exploration
        if self.frame_count < self.epsilon_random_frames or self.epsilon > np.random.rand(1)[0]:
            # Take random action
            # action = np.random.choice(self.action_size) <- на CPU работает медленнее
            action = random.sample(range(self.action_size), 1)[0]
        else:
            state_tensor = self._sample_transformer(state)
            # Take best action
            action_probs = self.model(state_tensor, training=False)
            # action = tf.argmax(action_probs[0]).numpy() <- на CPU работает медленнее
            action = np.argmax(action_probs[0])

        # Decay probability of taking random action
        if self.epsilon > self.epsilon_min:
            self.epsilon -= self.epsilon_decay

        return action

    def replay(self):
        states, actions, rewards, next_states, dones = self.replay_buffer.sample(self.batch_size)
        states = self._batch_transformer(np.array(states))
        next_states = self._batch_transformer(np.array(next_states))

        # Build the updated Q-values for the sampled future states
        # Use the target model for stability
        future_rewards = self.model_target(next_states)
        # Q value = reward + discount factor * expected future reward
        updated_q_values = rewards + self.gamma * tf.reduce_max(future_rewards, axis=1)

        # If final frame set the last value to -1
        updated_q_values = updated_q_values * (1 - dones) - dones

        # Create a mask so we only calculate loss on the updated Q-values
        masks = tf.one_hot(actions, self.action_size)

        with tf.GradientTape() as tape:
            # Train the model on the states and updated Q-values
            q_values = self.model(states)
            # Apply the masks to the Q-values to get the Q-value for action taken
            q_action = tf.reduce_sum(tf.multiply(q_values, masks), axis=1)
            # Calculate loss between new Q-value and old Q-value
            loss = self.loss_function(updated_q_values, q_action)
            self.episode_loss.append(loss)

        # Backpropagation
        grads = tape.gradient(loss, self.model.trainable_variables)
        self.optimizer.apply_gradients(zip(grads, self.model.trainable_variables))

    def train(self, goal_reward=None, max_frames=None):

        while True:
            self.frame_count += 1

            if self.new_episode:
                self.new_episode = False
                self.episode_start = time.time()
                self.episode_loss = []
                self.episode_reward = 0
                self.episode_count += 1
                self.state = self.env.reset()

            # env.render()
            action = self.act(self.state)
            next_state, reward, done, _ = self.env.step(action)
            self.episode_reward += reward

            # Save actions and states in replay buffer
            self.replay_buffer.push(self.state, action, reward, next_state, done)
            self.state = next_state

            # Update every N frame and batch size is enough
            if len(self.replay_buffer) > self.batch_size and self.frame_count % self.update_after_actions == 0:
                self.replay()

            # Update target network every N episodes
            if self.frame_count % self.update_target_network == 0:
                self.model_target.set_weights(self.model.get_weights())
                message = f"{datetime.now().strftime('%H:%M:%S')} Running reward: {self.running_reward:<8.2f} " \
                          f"at episode {self.episode_count:<4} | frame {self.frame_count:<6} | " \
                          f"eps: {self.epsilon:<4.2f} | Running loss: {self.running_loss:.5f}"
                print(message)

            if done:
                # episode is done
                self.new_episode = True

                # Update running reward to check condition for solving
                self.episode_reward_history.append(self.episode_reward)
                self.running_reward = np.mean(self.episode_reward_history)

                self.episode_loss_history.append(np.mean(self.episode_loss))
                self.running_loss = np.mean(self.episode_loss_history)

                break
                # Stop criteria
                # if goal_reward is not None and self.running_reward >= goal_reward:  # Condition to consider the task solved
                #    print(f"Done with reward {self.running_reward} at frame {self.frame_count}")
                #    break

            if max_frames is not None and self.frame_count >= max_frames:
                break

    def get_config(self):
        keys = self.__dict__.keys()
        config = {}
        for key in keys:
            if key in self.EXPORT_FORBIDDEN_ATTRIBUTES:
                continue
            else:
                config[key] = getattr(self, key)
        return config

    def load_config(self, config):
        keys = config.keys()
        for key in keys:
            if hasattr(self, key):
                setattr(self, key, config[key])
            else:
                print(f"Key {key} not found in agent")


In [13]:
random.seed(seed_value)

core_train = core_constructor.get_core(core_config)
env = TradeEnv(core_train, dpf_train, alias=ALIAS, log=False, log_obs=False)

model = create_q_model(env)
model_target = create_q_model(env)
agent = DQNAgent(env, model, model_target)

agent.epsilon_greedy_frames = 2000
agent.epsilon_random_frames = int(0.05 * agent.epsilon_greedy_frames)
agent.max_memory_length = int(1.0 * agent.epsilon_greedy_frames)
agent.max_steps_per_episode = 50000
agent.gamma = 0.95
agent.epsilon_min = 0.01
agent.batch_size = 30
agent.update_after_actions = 4
agent.update_target_network = 1000
agent.loss_function = tf.keras.losses.Huber() #tf.keras.losses.MeanSquaredError()
agent.optimizer = Adam(learning_rate=0.0005, clipnorm=0.001)    #Adam(learning_rate=learning_rate) RMSprop(learning_rate=learning_rate) SGD(learning_rate=learning_rate)


tp = TrainPlot4()
core_test = core_constructor.get_core(core_config)
tm = TrainManager(agent, core_test, dpf_test, tp, alias=ALIAS)

In [14]:
tp.init_plot(width=1000, height=800)
tp.update_plot(tm.history)

FigureWidget({
    'data': [{'legendgroup': '1',
              'line': {'color': '#109618', 'width': 1},
              'mode': 'lines',
              'name': 'Train',
              'type': 'scatter',
              'uid': '952f393c-9a3a-489a-a388-c2c9566e6ec3',
              'xaxis': 'x',
              'yaxis': 'y'},
             {'legendgroup': '1',
              'line': {'color': '#FF9900', 'width': 1},
              'mode': 'lines',
              'name': 'Test',
              'type': 'scatter',
              'uid': 'afc005a4-1022-4ddd-a882-608ba3c69f5e',
              'xaxis': 'x',
              'yaxis': 'y'},
             {'legendgroup': '2',
              'line': {'color': '#D62728', 'width': 1},
              'mode': 'lines',
              'name': 'Train',
              'type': 'scatter',
              'uid': '5b5db21e-914e-4484-b737-fb458eabc23c',
              'xaxis': 'x2',
              'yaxis': 'y3'},
             {'legendgroup': '2',
              'line': {'color': '#FF9900'

In [15]:
tm.go(max_frames=10000, test_every=100, snapshot_every=500000, update_plot_every=100, save_since=0.06)

23:38:17 Running reward: -89.90   at episode 11   | frame 1000   | eps: 0.50 | Running loss: 0.77144
23:38:31 Running reward: -61.85   at episode 21   | frame 2000   | eps: 0.01 | Running loss: 0.67022
23:38:47 Running reward: -41.73   at episode 31   | frame 3000   | eps: 0.01 | Running loss: 0.54043
23:39:02 Running reward: -12.13   at episode 41   | frame 4000   | eps: 0.01 | Running loss: 0.30205
23:39:20 Running reward: -1.10    at episode 52   | frame 5000   | eps: 0.01 | Running loss: 0.10298
23:39:38 Running reward: -1.53    at episode 62   | frame 6000   | eps: 0.01 | Running loss: 0.02445
23:39:54 Running reward: -0.03    at episode 72   | frame 7000   | eps: 0.01 | Running loss: 0.02006
23:40:10 Running reward: -5.37    at episode 82   | frame 8000   | eps: 0.01 | Running loss: 0.04874
23:40:26 Running reward: -1.87    at episode 92   | frame 9000   | eps: 0.01 | Running loss: 0.07969
23:40:41 Running reward: -0.13    at episode 103  | frame 10000  | eps: 0.01 | Running loss

## Итоги
Конфиг из предыдущей задачи не справился за 10к фреймов. 

- Сделал сеть шире 8х8 -> 32х32: ОК: 8к, 0.9к, 9.4к, 9.6к
- Добавил 3 слой на 32 нейрона : ОК 1.3к, 7.4к, 1.1к, 1.1к

По итогу работает таже схема - сеть погрубже и пошире и алгоритм справляется с задачей более уверенно.
